# Invalide model - 415passiv i liv-invalid-død model
Forklaring: Grundform 415 er en ophørende Invaliderente\
Author:          Peter Pommergård Lind\
Email:           pli@pfa.dk\
Encoding:        utf-8\
CITATIONS: Edlund MVS og PFA Tekniske grundlag 2019

In [7]:
###########
# Libraries
###########
import pandas
import numpy as np

In [8]:
#############
## Functions
#############
# Integration
def laplace(a,b,f):
  integral = 0
  if a==b:
    return 0
  elif b==(a+1):
    integral +=f(a)/2
    integral +=f(b)/2
  else:
    integral +=f(a)/2
    integral +=f(b)/2
    for i in range(a+1,b):
      integral += f(i)
  return integral

#dødsintensitet
def muAd(alder,år):
  "Simple interpolation of muAD by rounding age to nearest int"
  if (alder%1==0):
    mu = intensitet[alder] * (1-trend[alder])**(år-2017)
  else:
    alder = round(alder)
    mu = intensitet[alder] * (1-trend[alder])**(år-2017)
  return(mu)
#invalideintensitet
def muAI(alder):
    return 0.0004+10**(5.26 + 0.048 * alder -10)

In [9]:
############
# Parameters
###########
n = 64 #pensionsalder
muData = pandas.read_excel(r'./data/muAD.xlsx')
intensitet = muData.Intensitet
trend = muData.Forbedring
år=2020
alder = 39
i=0.00

# Simpel liv-invaid-død model
415 passiv med u17 dødeligheder og u10 invalideintensiteter fra PFA tekniske grundlag. Passiv 415 regnes som aktiv uden præmiefritagelse fratrukket aktiv med præmiefritagelse:
$$Passiv415=aktivUPræmie - aktivMPræmie$$

## Aktiv uden præmiefritagelse (aktivUPræmie)
Vi antager personen er født 1/1/1981 og har pensionsalder 64. Derudover er beregningstidspunktet den 1/1/2020.\
$$ V(t)=\int_t^{64}\exp(-\int_t^s r+\mu d\tau) ds $$

In [2]:
def aktivUPræmie(g,n,år, r):
  """[Find the expected contributions to "opsat livrente"]
  Args:
      g ([int]): [alder ved beregningsdato]
      n ([int]): [sidste udbetalingsdato]
      år ([int]): year of udbetalingsstart
      r ([float]): [interest rate minus "sikkerhedstillæg"]
  """
  def f1(alder):
    # Inner integrand
    return (r+muAd(alder,år+(alder-g))) 
  def f2(tau):
    #second integrand
    return np.exp(-laplace(g,tau,f1))
  return laplace(g,n,f2)

print("aktiv uden præmiefritagelse", aktivUPræmie(g=39,n=64,år=2020, r=0))

NameError: name 'laplace' is not defined

## Aktiv med præmiefritagelse (aktivMPræmie)
Vi antager personen er født 1/1/1981 og har pensionsalder 64. Derudover er beregningstidspunktet den 1/1/2020.\
$$ V(t)=\int_t^{64}\exp(-\int_t^s r+\mu+ \nu d\tau) ds $$

In [10]:
def aktivMPræmie(g,n,år, r):
  """[Find the expected contributions to "opsat livrente"]
  Args:
      g ([int]): [alder ved beregningsdato]
      n ([int]): [sidste udbetalingsdato]
      år ([int]): year of udbetalingsstart
      r ([float]): [interest rate minus "sikkerhedstillæg"]
  """
  def f1(alder):
    # Inner integrand
    return (r+muAI(alder) +muAd(alder,år+(alder-g))) 
  def f2(tau):
    #second integrand
    return np.exp(-laplace(g,tau,f1))
  return laplace(g,n,f2)

print("aktiv Med præmiefritagelse", aktivMPræmie(g=39,n=64,år=2020, r=0))

aktiv uden præmiefritagelse 24.70238518644664
aktiv Med præmiefritagelse 23.33672203962295
Passiv 415 1.3656631468236924


# Passiv415

In [1]:
def passiv415(g,n,år,r):
    return aktivUPræmie(g,n,år,r)-aktivMPræmie(g,n,år,r)

print("Passiv 415", passiv415(39,64,2020,0))

NameError: name 'aktivUPræmie' is not defined